# Cloud Computing Fundamentals Practical
## Running an application on EC2
### By: Eric Meissner

## Prerequisites 
* AWS Account created
* Boto3
* AWS CLI - https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux.html

``` pip install boto3```

## S3
* Make a bucket
* Upload some data and a training.py file to that bucket

## EC2
* Spin up an instance from SDK, explain AMIs (Virtual Machine images), networking / firewall, availability zones + regions (here or later?, the logic behind them)
* SSH to EC2 instance, SCP over the training.py file
* Spin up Jupyter notebook on instance + access from local machine
* Spin up the training application and call it from your local machine


### 0. Login to the AWS Console 

### 1. Create new EC2 instance (from the console)
1. Click on (in the top left) Services -> EC2.
2. Click on "Instances" in the left sidebar, then "Launch instances" on the right side.
3. This brings us to a page where you select what AMI to use as the base for the machine. An AMI is essentially packaged base operating system, and may have some additional packages installed depending on the AMI you choose. 
  1. Let's choose the ```Ubuntu Server 18.04 LTS (HVM), SSD Volume Type - ami-0bcc094591f354be2``` because it's free tier eligible. 
  2. The Deep Learning AMIs come pre-installed with drivers such as NVidia GPU drivers and common DL frameworks such as Tensorflow, Pytorch, and MXNet. The "Deep Learning Base" AMI sets come with only the additional drivers installed, but not the frameworks. These are useful starting points for most applications, as configuring drivers can be somewhat of a hassle.
4. The next screen brings us to the choice of hardware we want to run. Typically, you'll choose this based on what the application's needs are (such as whether you need a GPU, high memory, etc.) In this example, we'll choose a very small instances, the ```t2.micro``` because it's eligible for free tier status. 
  1. Note: this is a helpful website for looking at the stats of EC2 instances https://www.ec2instances.info/.
5. Click on "Configure Instance Details" next. 
  1. We don't need to change anything here, but notice that the instance is created inside your account's default "network" or VPC. All accounts start with a default VPC, and all instances live within some VPC and this controls what network traffic goes into and out of the instance. An important aspect of subnets is defining which Availability Zone an instance is launched into. 
6. Click "Add Storage". Here you can increase or decrease the amount of block storage (EBS) your instance will have when spun up. You can change this later, so we can leave it at default for now.
7. Click through "Add Tags" (they are just key-value pair meta-data additions for the instance to help you manage them), and click on "Configure Security Groups".
  1. Remember that Security Groups essentially define firewall rules for instances inside of that security group. If you don't place an EC2 instance into a security group, anyone can access this machine if they have the correct PEM key (which we get in the next step.) For most production machines, you'll want access to be tightly restricted, typically to only other machines within the subnet and if they're serving web traffic then only full access rules on the HTTP/HTTPS ports.
  2. For today's uses though, we'll simply add a security group that allows SSH access from any address. This should already be populated when you've selected the "Create New Security Group" option. SSH's port is 22, and we allow access from 0.0.0.0/0, i.e. any address. 
8. Click through "Review and Launch" and look over the information on the page to make sure it's alright. It will show a warning up top regarding our security groups, this is fine. 
9. When you click Launch, it will prompt you to select or create a keypair. If you already have one, feel free to use that, otherwise create a keypair with a useful name (often people have a personal keypair for their instances ("bob_keypair"), and then separate ones for collective instances like "ml_training_instances_keypair".) You'll then need to download and **not lose** this keypair, as you can't get it back and any instance launched using the keypair will require you to have it to access via SSH.
10. Once you've downloaded your new keypair though, launch the instance! You can go back to the instances page to see the instance launching (they take a few minutes to initialize.)

### Create an IAM User to login from Boto3 / console
IAM controls access to AWS resources. There are IAM users and IAM roles, and we'll need to create a user in order to access our AWS account from the command line. IAM Roles are *assumed* by AWS resources when they run that allow more natural access to other resources. I.e. an EC2 host can assume an IAM role to access S3 buckets without needing user keys explicitly setup.

1. From the console, Services -> IAM.
2. Add User -> Fill in a username -> Tick the "Programmatic box", you won't need the "AWS Console" one. -> Next: Permissions
3. Click create group (ex. name: 'cloud_computing_module'), so that you can create new users in the future with the same policies already setup. Initially, we will want to create an S3 bucket and add to it from the command line, so add the ```AmazonS3FullAccess``` policy.
4. Click through tags and create your new user!
5. When you get to the "Success!" screen, click "download .csv" to download the credential pair. This consists of an Access Key and an Secret Access Key.
6. From a terminal, run ```aws configure``` and input the access key and secret access keys when prompted.
7. You're good to go now!

### Setup S3 Bucket with Boto3
Boto3 is the Python library for accessing and managing AWS services. Below I give a basic use case for creating an S3 bucket. The documentation is really good for it, so just google whatever you're trying to do with it.

In [2]:
!pip install boto3

     |████████████████████████████████| 129 kB 1.5 MB/s eta 0:00:01


In [5]:
import boto3 
session = boto3.session.Session()
s3_connection = session.resource("s3")

In [14]:
# Make sure to create a bucket with an appropriate name
bucket_name = "cloud-computing-module"

In [15]:
s3_connection.create_bucket(Bucket=bucket_name)

s3.Bucket(name='cloud-computing-module')

In [17]:
bucket = s3_connection.Bucket(bucket_name)

In [12]:
# the location of the object within the bucket
key = "data.txt"
# the content of the object
result = "Hello World"


In [13]:
# Write the content to the specific location in the given bucket
bucket.Object(key).put(Body=result)

{'ResponseMetadata': {'RequestId': 'C8EE7F58E7421C46',
  'HostId': 'V9EK6ODnBxO5mbXPypRgWgVJ+JEQhB2WRxDvfRT5OsApd5fF/4Q58m4CEd1hG6/1lIK04rYXGnY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'V9EK6ODnBxO5mbXPypRgWgVJ+JEQhB2WRxDvfRT5OsApd5fF/4Q58m4CEd1hG6/1lIK04rYXGnY=',
   'x-amz-request-id': 'C8EE7F58E7421C46',
   'date': 'Mon, 31 Aug 2020 15:40:37 GMT',
   'etag': '"b10a8db164e0754105b7a99be72e3fe5"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"b10a8db164e0754105b7a99be72e3fe5"'}

### SSH to EC2 instance and run training script

To SSH to your instance run something like the following using the keypair you created when launching your instance:

```bash
ssh -i <path/to/keypair.pem> ubuntu@<public_ip_of_your_ec2_instance>
```

You can find the public IP or dnsname of your instance by clicking on it from the console and finding the information.

Also note that if you used something other than a Ubuntu machine, you may need to have a different prefix ```ec2-user@<IP>``` for Amazon Linux for example.